In [ ]:
import os
from pymongo import MongoClient
from langchain.vectorstores import MongoDBAtlasVectorSearch
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from dotenv import load_dotenv
from langchain.document_loaders.mongodb import MongodbLoader
import nest_asyncio

nest_asyncio.apply()
load_dotenv()

In [ ]:
loader = MongodbLoader(
    connection_string=os.getenv('MONGODB_URI'),
    db_name=os.getenv('MONGODB_DB'),
    collection_name=os.getenv('MONGODB_COLL'),
    filter_criteria={},
    field_names=["title","plot"]
)
docs = loader.load()

len(docs)

In [ ]:
client = MongoClient(os.getenv('MONGODB_URI'))
collection = client[os.getenv('MONGODB_DB')][os.getenv('MONGODB_VECTOR_COLL_LANGCHAIN')]

vector_search = MongoDBAtlasVectorSearch.from_documents(
  documents=docs,
  embedding=OpenAIEmbeddings(openai_api_key=os.getenv('OPENAI_API_KEY')),
  collection=collection,
  index_name=os.getenv('MONGODB_VECTOR_INDEX'))

In [ ]:

# perform a similarity search on the ingested documents
prompt='Young Ones Set in the future when water is hard to find, a teenage boy sets out to protect his family and survive'
docs = vector_search.similarity_search(prompt,1)
for doc in docs:
    print(doc.page_content)

# perform a similarity search with score on the ingested documents

docs_with_score = vector_search.similarity_search_with_score(query=prompt,k=1)
print(" ")
print("Result with Scores")

for doc, score in docs_with_score:
    print(doc.page_content)
    print(score)